In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torchinfo import summary
from config import en_id_model as mtconf, get_default_device
from dataset import get_tokenizers
from model import build_model
from train import train_model
from utils import TrainCheckpoint, EarlyStopping, ReduceLROnPlateau, TrainingCallback

c:\Users\Nizwa\miniconda3\envs\ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
t_src, t_tgt = get_tokenizers(mtconf, ds_train=None, force_retrain_tokenizer=False)
model = build_model(mtconf, t_src, t_tgt)

summary(
    model,
    input_data=[
        torch.randint(0, t_src.get_vocab_size(), (mtconf.batch_size, mtconf.seq_len)),  # encoder_input
        torch.randint(0, t_tgt.get_vocab_size(), (mtconf.batch_size, mtconf.seq_len)),  # decoder_input
        torch.ones(mtconf.batch_size, 1, 1, mtconf.seq_len, dtype=torch.int),  # encoder_mask
        torch.ones(mtconf.batch_size, 1, mtconf.seq_len, mtconf.seq_len, dtype=torch.int),  # decoder_mask
    ],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    depth=10,
    row_settings=["var_names"]
)

tokenizer exist, getting from: .output\tokenizer_en.json
tokenizer exist, getting from: .output\tokenizer_id.json


Layer (type (var_name))                                           Input Shape               Output Shape              Param #                   Trainable
Transformer (Transformer)                                         [64, 50]                  [64, 50, 30000]           --                        True
├─InputEmbedding (src_embed)                                      [64, 50]                  [64, 50, 256]             --                        True
│    └─Embedding (embedding)                                      [64, 50]                  [64, 50, 256]             7,680,000                 True
├─PositionalEncoding (src_pos)                                    [64, 50, 256]             [64, 50, 256]             --                        --
│    └─Dropout (dropout)                                          [64, 50, 256]             [64, 50, 256]             --                        --
├─Encoder (encoder)                                               [64, 50, 256]             [64, 50, 256]

In [3]:
get_default_device()

device(type='cuda')

In [4]:
%load_ext autoreload
%autoreload 2

tcp = TrainCheckpoint(mtconf.model_output)
es = EarlyStopping(patience=5)
rlr = ReduceLROnPlateau(factor=0.5, patience=2, cooldown=2)
callback = TrainingCallback(checkpoint=tcp, early_stop=es, reduce_lr=rlr)

history = train_model(mtconf, callback, preload=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


total sentence pair for training = 1000000
tokenizer exist, getting from: .output\tokenizer_en.json
tokenizer exist, getting from: .output\tokenizer_id.json
max length of source sentence: 579
max length of target sentence: 535


Filter: 100%|██████████| 2000/2000 [00:00<00:00, 9618.73 examples/s] 


total sentence pair for training after filtering = 993856


epoch 1: 100%|██████████| 15529/15529 [13:17<00:00, 19.47it/s, loss=3.832]


...

Epoch 17 - 0:12:36.8 | train_loss=2.687878 | val_loss=3.188818 | CER=0.6042919158935547 | WER=1.3137699365615845 | BLEU=0.08063607662916183
Reducing LR in the next epoch from 0.000050 to 0.000025
metrics did not improve from 3.168913




epoch 18: 100%|██████████| 15529/15529 [12:50<00:00, 20.15it/s, loss=3.071]


Epoch 18 - 0:13:11.3 | train_loss=2.650518 | val_loss=3.190245 | CER=0.712478518486023 | WER=1.5928415060043335 | BLEU=0.06702635437250137
metrics did not improve from 3.168913
Early stopping, no improvement in the last 5 epochs




epoch 19:  55%|█████▍    | 8497/15529 [06:57<05:45, 20.34it/s, loss=2.832]


KeyboardInterrupt: 